# Hyperameter tuning (of model using simulated and experimental data) with Keras tuner
### target is validation loss, best model is saved, tunable parammeters are number of hidden layers, number of units in each layer, dropout, batch size

In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math
from LSTMutils import MeanVarianceLogLikelyhoodLoss
from LSTMutils import LSTMnetwork
from sklearn.model_selection import train_test_split
import keras_tuner as kt

np.random.seed(42)
tf.random.set_seed(42)

SequenceLength = 250
validation_split = 0.25
NumEpochs = 2000
test_split = 0.2

df_experimental = pd.read_csv(r"../TrainingData/ExperimentalTrainingSet.csv",sep=',',header=None)
time = df_experimental.iloc[0,1:]/3600
idealTime = np.append([0,0], time.iloc[2:,].values)
ExperimentalLabels = df_experimental.iloc[1:,0]
df_experimental = df_experimental.iloc[1:,1:]

df_simulated = pd.read_csv(r"../TrainingData/SimulatedTrainingSet10000.csv",sep=',',header=0)
df_simulated = df_simulated.iloc[:,1:]
SimulatedLabels = df_simulated.iloc[1:,0]

df_train_experimental, df_test = train_test_split(df_experimental, test_size=test_split, train_size=1-test_split, random_state=42, shuffle=True, stratify=ExperimentalLabels)

ExperimentalTrainLabels = df_train_experimental.iloc[:,0]

df_train_experimental, df_val = train_test_split(df_train_experimental, test_size=validation_split, train_size=1-validation_split, random_state=42, shuffle=True, stratify=ExperimentalTrainLabels)

df_simulated.columns = range(1,df_simulated.columns.size+1)

df_train = pd.concat([df_train_experimental, df_simulated], ignore_index=True)

# normalise time series data
min_value, max_value = df_train.min().min(), df_train.max().max()
df_norm_train = (df_train - min_value)/(max_value - min_value)
df_norm_test = (df_test - min_value)/(max_value - min_value)
df_norm_val = (df_val - min_value)/(max_value - min_value)

df_norm_train = df_norm_train.sample(frac=1)
df_norm_val = df_norm_val.sample(frac=1)
    
X_train = df_norm_train.iloc[:,:SequenceLength].values
y_train = df_norm_train.iloc[:,SequenceLength-1].values
X_train = np.expand_dims(X_train, 2)
y_train = np.broadcast_to(y_train[:,None], (y_train.shape[0],SequenceLength))
y_train = np.expand_dims(y_train, 2)

X_val = df_norm_val.iloc[:,:SequenceLength].values
y_val = df_norm_val.iloc[:,SequenceLength-1].values
X_val = np.expand_dims(X_val, 2)
y_val = np.broadcast_to(y_val[:,None], (y_val.shape[0],SequenceLength))
y_val = np.expand_dims(y_val, 2)

#Implementation informed by https://github.com/keras-team/keras-tuner/issues/122

class LSTMHyperModel(kt.HyperModel):
    
    def build(self,hp):
        model = keras.models.Sequential(keras.layers.LSTM(hp.Choice("layer_1_units", [5,10,20,50,100,200,500,1000,2000]), input_shape=(SequenceLength,1), return_sequences=True))#, kernel_regularizer=keras.regularizers.L2(hp.Choice("L2Regularizer1", [0., 1e-4]))))
        if hp.Boolean("layer_1_dropout"):
            model.add(keras.layers.Dropout(rate=0.25))
        for i in range(hp.Int("LSTM Layers", min_value = 1, max_value = 8)):
            model.add(keras.layers.LSTM(hp.Choice(f"layer_{i+2}_units", [5,10,20,50,100,200,500,1000,2000]), return_sequences=True))#, kernel_regularizer=keras.regularizers.L2(hp.Choice(f"L2Regularizer{i+2}", [0., 1e-4]))))
            if hp.Boolean(f"layer_{i+2}_Dropout"):
                model.add(keras.layers.Dropout(rate=0.25))
        #learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
        model.add(keras.layers.LSTM(2, activation='softplus',return_sequences=True))

#         optimizer = hp.Choice("optimizer", ["adam","sgd"])
        
        model.compile(optimizer="adam",loss = MeanVarianceLogLikelyhoodLoss)
    
        return model

    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size",[32,64,128,256,512,1024]),**kwargs)

tuner = kt.RandomSearch(#kt.BayesianOptimization(#kt.Hyperband(#(#
    LSTMHyperModel(),
    objective='val_loss',
    overwrite = False,#True,#
    #beta=5,
    #factor = 3,
    #hyperband_iterations = 1,
    #max_epochs = 100,
    max_trials = 300,
    seed = 42,
    directory = "../Models/SimulatedAndExperimentalModelTuner")
        
tuner.search(X_train, y_train, epochs=NumEpochs, validation_data=(X_val,y_val), callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=50),keras.callbacks.TerminateOnNaN()])
best_model = tuner.get_best_models()[0]
best_model.summary()
best_model.save("../Models/SimulatedAndExperimentalModelTuner")

Trial 120 Complete [00h 19m 41s]
val_loss: -3.620673179626465

Best val_loss So Far: -3.795105457305908
Total elapsed time: 1d 01h 31m 32s

Search: Running Trial #121

Value             |Best Value So Far |Hyperparameter
200               |100               |layer_1_units
5                 |1                 |LSTM Layers
200               |500               |layer_2_units
1024              |32                |batch_size
500               |100               |layer_3_units
20                |100               |layer_4_units
100               |50                |layer_5_units
500               |500               |layer_6_units
20                |5                 |layer_7_units
50                |20                |layer_8_units
False             |False             |layer_1_dropout
False             |True              |layer_2_Dropout
True              |False             |layer_3_Dropout
False             |True              |layer_4_Dropout
False             |False             |layer_5_Dr

2023-11-11 23:21:26.720236: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.38GiB (rounded to 2560000000)requested by op CudnnRNN
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-11 23:21:26.720318: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2023-11-11 23:21:26.720330: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 213, Chunks in use: 213. 53.2KiB allocated for chunks. 53.2KiB in use in bin. 3.1KiB client-requested in use in bin.
2023-11-11 23:21:26.720337: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 25, Chunks in use: 25. 13.5KiB allocated for chunks. 13.5KiB in use in bin. 9.7KiB client-requested in use in bin.
2023-11-11 23:21:26.720342: 

InternalError: Graph execution error:

Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 100, 500, 1, 250, 1024, 500] 
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm_5/PartitionedCall]] [Op:__inference_train_function_5729565]

11cde32800 of size 3328 next 6233
2023-11-11 23:21:26.721398: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 7f11cde33500 of size 3840 next 1275
2023-11-11 23:21:26.721403: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 7f11cde34400 of size 1792 next 2525
2023-11-11 23:21:26.721410: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 7f11cde34b00 of size 2560 next 2434
2023-11-11 23:21:26.721416: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 7f11cde35500 of size 256 next 5579
2023-11-11 23:21:26.721421: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 7f11cde35600 of size 2048 next 2142
2023-11-11 23:21:26.721426: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 7f11cde35e00 of size 256 next 385
2023-11-11 23:21:26.721431: I tensorflow/core/common_runtime/bfc_allocator.cc:1083] InUse at 7f11cde35f00 of size 208384 next 5858
2023-11-11 23:21:26.721437: I tensorflow/core/common_runtime/bfc

In [4]:
tuner.results_summary(20)

Results summary
Results in ../Models/SimulatedAndExperimentalModelTuner/untitled_project
Showing 20 best trials
Trial summary
Hyperparameters:
layer_1_units: 100
LSTM Layers: 1
layer_2_units: 500
batch_size: 32
layer_3_units: 100
layer_4_units: 100
layer_5_units: 50
layer_6_units: 500
layer_7_units: 5
layer_8_units: 20
layer_1_dropout: False
layer_2_Dropout: True
layer_3_Dropout: False
layer_4_Dropout: True
layer_5_Dropout: False
layer_6_Dropout: False
layer_7_Dropout: False
layer_8_Dropout: True
layer_9_units: 5
layer_9_Dropout: True
Score: -3.795105457305908
Trial summary
Hyperparameters:
layer_1_units: 100
LSTM Layers: 4
layer_2_units: 50
batch_size: 64
layer_3_units: 10
layer_4_units: 100
layer_5_units: 100
layer_6_units: 50
layer_7_units: 50
layer_8_units: 10
layer_1_dropout: False
layer_2_Dropout: False
layer_3_Dropout: False
layer_4_Dropout: True
layer_5_Dropout: False
layer_6_Dropout: False
layer_7_Dropout: False
layer_8_Dropout: False
layer_9_units: 50
layer_9_Dropout: False
S